# Importing Python modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('CustomerData.csv')

# Pipeline Model

In [3]:
df.head()

,CustomerID,City,NoOfChildren,MinAgeOfChild,MaxAgeOfChild,Tenure,FrquncyOfPurchase,NoOfUnitsPurchased,FrequencyOFPlay,NoOfGamesPlayed,NoOfGamesBought,FavoriteChannelOfTransaction,FavoriteGame,TotalRevenueGenerated
0,1001,1,2,3,8,210,11,11,2344,108,10,Uniform,Uniform,107.51
1,1002,1,2,3,6,442,20,20,245,22,7,Favorite,Uniform,382.40
2,1003,1,4,3,5,424,18,18,1059,130,18,Favorite,Uniform,135.01
3,1004,1,1,6,6,261,11,9,365,34,11,Favorite,Uniform,125.00
4,1005,1,3,6,9,422,44,31,1066,102,44,Uniform,Uniform,335.05


- Step 1: Split the data into training and testing sets
- Step 2: TensorFlow objects using tf.data
- Step 3: Feature column
- Step 4: Model

In [4]:
# split the data into training and testing
test_df = df.sample(frac=0.2)
train_df = df.drop(test_df.index)

In [6]:
test_df.shape, train_df.shape

((642, 14), (2567, 14))

# Create TensorFlow objects

In [7]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [8]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow import feature_column
# tensorboard
from tensorflow.keras.callbacks import TensorBoard

In [9]:
def dataframes_to_datasets(datafram):
    df = datafram.copy()
    labels = df.pop('TotalRevenueGenerated')

    # convert into datasets
    ds = tf.data.Dataset.from_tensor_slices((dict(df),labels))
    
    # shuffle
    ds = ds.shuffle(buffer_size= len(df))
    
    return ds

In [10]:
train_ds = dataframes_to_datasets(train_df)
test_ds = dataframes_to_datasets(test_df)

In [11]:
train_ds = train_ds.batch(32)
test_ds = test_ds.batch(32)

In [17]:
example_data = next(iter(test_ds))

In [18]:
example_data

({'CustomerID': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([4115, 3935, 1429, 2440, 1794, 3619, 1596, 1252, 1031, 2517, 3333,
         1963, 1311, 1579, 2927, 2616, 3802, 3933, 2386, 1496, 1385, 1023,
         1829, 3976, 3661, 3091, 1559, 3105, 1556, 3370, 3044, 3636],
        dtype=int64)>,
  'City': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 2, 1, 1, 1, 1, 1, 1], dtype=int64)>,
  'NoOfChildren': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([3, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 3, 1, 2, 2, 2, 1, 1, 6, 1, 2, 1,
         3, 3, 1, 2, 2, 2, 1, 1, 2, 2], dtype=int64)>,
  'MinAgeOfChild': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([6, 4, 3, 8, 5, 6, 4, 5, 7, 3, 2, 2, 6, 5, 3, 4, 7, 4, 5, 9, 6, 8,
         3, 3, 4, 5, 4, 4, 9, 4, 5, 3], dtype=int64)>,
  'MaxAgeOfChild': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([12,  6,  6,  8,  9,  6,  8,  5,  7,  5,  4,  6,  6,

# Feature Layer

['NoOfGamesBought', 'FrquncyOfPurchase', 'NoOfUnitsPurchased', 'fct_Favorite', 'MinAgeOfChild', 'NoOfChildren', 'FrequencyOFPlay', 'NoOfGamesPlayed', 'city_1']

In [20]:
# categorical
fct = feature_column.categorical_column_with_vocabulary_list('FavoriteChannelOfTransaction', ['Favorite'])
fct_fav = feature_column.indicator_column(fct)

city = feature_column.categorical_column_with_vocabulary_list('City', [1])
city_1 = feature_column.indicator_column(city)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [21]:
df.columns

Index(['CustomerID', 'City', 'NoOfChildren', 'MinAgeOfChild', 'MaxAgeOfChild',
       'Tenure', 'FrquncyOfPurchase', 'NoOfUnitsPurchased', 'FrequencyOFPlay',
       'NoOfGamesPlayed', 'NoOfGamesBought', 'FavoriteChannelOfTransaction',
       'FavoriteGame', 'TotalRevenueGenerated'],
      dtype='object')

In [22]:
# numerical columns
num_col = ['NoOfGamesBought', 'FrquncyOfPurchase', 'NoOfUnitsPurchased',
           'MinAgeOfChild', 'NoOfChildren', 'FrequencyOFPlay', 'NoOfGamesPlayed']
featured = []
for i in range(len(num_col)):
    featured.append(feature_column.numeric_column(num_col[i]))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [24]:
featured.extend([fct_fav, city_1])

In [26]:
featured

[NumericColumn(key='NoOfGamesBought', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='FrquncyOfPurchase', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NoOfUnitsPurchased', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='MinAgeOfChild', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NoOfChildren', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='FrequencyOFPlay', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NoOfGamesPlayed', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='FavoriteChannelOfTransaction', vocabulary_list=('Favorite',), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=Vocabulary

# Regression Model

In [27]:
model_reg = Sequential([
    layers.DenseFeatures(featured),
    layers.Dense(units=1)
])

# loss and optimizer
loss = tf.keras.losses.mean_squared_error
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile model
model_reg.compile(loss = loss, optimizer = optimizer)

In [28]:
tfb = TensorBoard(log_dir='final_reg_model')

In [29]:
history = model_reg.fit(train_ds, epochs=2000, validation_data=test_ds, callbacks=[tfb])

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Epoch 1/2000

81/81 [==============================] - 2s 8ms/step - loss: 912040.8125 - val_loss: 853737.8750
Epoch 2/2000
81/81 [==============================] - 0s 4ms/step - loss: 639149.8125 - val_loss: 599414.6250
Epoch 3/2000
81/81 [==============================] - 0s 4ms/step - loss: 440041.7188 - val_loss: 404641.7500
Epoch 4/2000
81/81 [==============================] - 0s 4ms/step - loss: 295671.4062 - val_loss: 272703.9375
Epoch 5/2000
81/81 [==============================] - 0s 3ms/step - loss: 194838.2656 - val_loss: 180337.4219
Epoch 6/2000
81/81 [==============================] - 0s 3ms/step - loss: 128115.4453 - val_loss: 121352.6641
Epoch 7/2000
81/81 [==============================] - 0s 4ms/step - loss

In [30]:
%load_ext tensorboard

In [32]:
%tensorboard --logdir final_reg_model

Reusing TensorBoard on port 6006 (pid 24292), started 0:00:07 ago. (Use '!kill 24292' to kill it.)

**Save Model**

In [33]:
tf.keras.models.save_model(model_reg, 'pipeline_model')

INFO:tensorflow:Assets written to: pipeline_model\assets


INFO:tensorflow:Assets written to: pipeline_model\assets


In [34]:
# restoring the model
model_load = tf.keras.models.load_model('pipeline_model')

In [35]:
model_reg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features (DenseFeatu  multiple                  0         
 res)                                                            
                                                                 
 dense (Dense)               multiple                  10        
                                                                 
Total params: 10 (40.00 Byte)
Trainable params: 10 (40.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model_reg.weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(9, 1) dtype=float32, numpy=
 array([[-1.8573083e+00],
        [ 1.4091269e+01],
        [ 4.1440092e-03],
        [ 9.4330273e+00],
        [ 9.1407704e-01],
        [ 2.2415123e+00],
        [-1.0840937e+01],
        [-5.5712681e-02],
        [ 9.7159519e+00]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(1,) dtype=float32, numpy=array([14.119957], dtype=float32)>]

In [37]:
y_pred_org = model_reg.predict(test_ds)

21/21 [==============================] - 0s 2ms/step


In [38]:
y_pred_org

array([[264.91672 ],
       [147.57669 ],
       [168.31653 ],
       [197.05508 ],
       [110.958855],
       [164.68811 ],
       [167.74976 ],
       [229.50253 ],
       [173.58716 ],
       [115.29569 ],
       [247.14435 ],
       [154.7952  ],
       [142.11856 ],
       [148.61603 ],
       [144.14273 ],
       [305.0797  ],
       [178.48828 ],
       [105.768776],
       [109.12568 ],
       [162.16632 ],
       [133.70978 ],
       [182.35065 ],
       [176.46338 ],
       [180.20416 ],
       [139.6307  ],
       [113.74771 ],
       [143.9632  ],
       [756.4597  ],
       [171.89478 ],
       [208.58926 ],
       [144.28574 ],
       [219.82803 ],
       [229.92569 ],
       [137.13547 ],
       [201.75922 ],
       [299.55908 ],
       [127.90934 ],
       [200.35767 ],
       [118.86952 ],
       [117.90356 ],
       [108.428474],
       [300.14462 ],
       [124.95837 ],
       [149.1456  ],
       [222.88321 ],
       [188.31879 ],
       [305.22018 ],
       [173.3

In [39]:
y_pred_load = model_load.predict(test_ds)

21/21 [==============================] - 0s 3ms/step


In [40]:
y_pred_load

array([[173.58716 ],
       [108.428474],
       [198.76733 ],
       [131.80365 ],
       [208.23358 ],
       [191.521   ],
       [120.763954],
       [140.45667 ],
       [112.1688  ],
       [155.52567 ],
       [166.96414 ],
       [176.01828 ],
       [152.01791 ],
       [139.6307  ],
       [197.05508 ],
       [102.59265 ],
       [ 94.98547 ],
       [118.940254],
       [132.52551 ],
       [256.9358  ],
       [152.9448  ],
       [170.49878 ],
       [178.47571 ],
       [117.90444 ],
       [172.61057 ],
       [142.48364 ],
       [358.25876 ],
       [ 86.328   ],
       [284.42023 ],
       [175.69748 ],
       [251.0794  ],
       [178.58655 ],
       [141.00482 ],
       [135.3934  ],
       [201.87827 ],
       [178.43729 ],
       [149.97351 ],
       [191.95856 ],
       [756.4597  ],
       [107.4282  ],
       [182.7248  ],
       [188.9241  ],
       [135.62048 ],
       [123.532906],
       [214.1752  ],
       [171.41977 ],
       [209.1406  ],
       [143.5